In [33]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import numpy as np

from keras.layers import Input, Dense, Activation, TextVectorization, Embedding, GlobalAveragePooling1D
from keras.models import Model
import tensorflow as tf

In [102]:
data_twitter = pd.read_csv("data_twitter.csv", engine='python', encoding = "ISO-8859-1").dropna()
data_twitter = data_twitter[data_twitter["sentiment"] != "neutral"]
data_twitter = data_twitter[["text", "sentiment"]]

data_imdb = pd.read_csv('data_imdb.csv')
data_imdb["text"] = data_imdb["review"]
data_imdb = data_imdb.drop("review", axis=1)

data = pd.concat([data_twitter, data_imdb])
data

,text,sentiment
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
6,2am feedings for the baby are fun when he is a...,positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [103]:
ohe = OneHotEncoder()

y_ohe = ohe.fit_transform(data['sentiment'].to_numpy().reshape((-1,1))).todense()

X_train, X_test, y_train, y_test = train_test_split(data['text'], y_ohe)

In [105]:
# Negative, Positive
print(y_ohe[0], list(data['sentiment'])[0], list(data['text'])[0])

[[1. 0.]] negative  Sooo SAD I will miss you here in San Diego!!!


In [108]:
vocab_size = 100000

def avg_embedding_softmax_model(vectorize_layer, vocab_size=vocab_size):
    input_layer = Input(shape=(1,), dtype=tf.string)
    x = input_layer
    x = vectorize_layer(x)
    x = Embedding(vocab_size, 2, name='projecao')(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(2, name='classificador')(x)
    x = Activation('softmax')(x)
    return Model(input_layer, x)

vectorize_layer = TextVectorization(output_mode='int', max_tokens=vocab_size, pad_to_max_tokens=True, output_sequence_length=200)
vectorize_layer.adapt(X_train)
clf = avg_embedding_softmax_model(vectorize_layer)
print(clf.summary())
clf.compile(loss='categorical_crossentropy', metrics=['accuracy'])
history = clf.fit(X_train, y_train, epochs=35, verbose=1) 
clf.evaluate(X_test, y_test)

2023-04-23 17:16:12.438221: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_12 (Text  (None, 200)              0         
 Vectorization)                                                  
                                                                 
 projecao (Embedding)        (None, 200, 2)            200000    
                                                                 
 global_average_pooling1d_5   (None, 2)                0         
 (GlobalAveragePooling1D)                                        
                                                                 
 classificador (Dense)       (None, 2)                 6         
                                                                 
 activation_7 (Activation)   (None, 2)                 0   

2023-04-23 17:16:15.696038: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1556/1556 [==============================] - 90s 58ms/step - loss: 0.6638 - accuracy: 0.6659
Epoch 2/35
1556/1556 [==============================] - 62s 40ms/step - loss: 0.5746 - accuracy: 0.7483
Epoch 3/35
1556/1556 [==============================] - 58s 37ms/step - loss: 0.4899 - accuracy: 0.7937
Epoch 4/35
1556/1556 [==============================] - 55s 36ms/step - loss: 0.4324 - accuracy: 0.8204
Epoch 5/35
1556/1556 [==============================] - 54s 35ms/step - loss: 0.3956 - accuracy: 0.8371
Epoch 6/35
1556/1556 [==============================] - 53s 34ms/step - loss: 0.3714 - accuracy: 0.8506
Epoch 7/35
1556/1556 [==============================] - 52s 34ms/step - loss: 0.3536 - accuracy: 0.8593
Epoch 8/35
1556/1556 [==============================] - 52s 33ms/step - loss: 0.3397 - accuracy: 0.8655
Epoch 9/35
1556/1556 [==============================] - 51s 33ms/step - loss: 0.3288 - accuracy: 0.8704
Epoch 10/35
1556/1556 [==============================] - 52s 33ms/step - lo

2023-04-23 17:51:58.645891: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


519/519 [==============================] - 16s 31ms/step - loss: 0.3624 - accuracy: 0.8580


[0.3623591661453247, 0.8579952716827393]

In [129]:
x_single = ['This is a sad sentence']
y_single = np.array([1])

y_pred_single = clf.predict(x_single)

1/1 [==============================] - 0s 25ms/step


In [130]:
positivity = y_pred_single[0][1]
positivity

0.20571427

In [137]:
clf.save('classifier')

INFO:tensorflow:Assets written to: classifier/assets
